# Day 15
## Part 1
~~I'm skipping this one for now, it looks annoying.~~ This still looks annoying but it's the only day I haven't done.

In [1]:
from pyrsistent import pmap, pset

from dataclasses import dataclass
import heapq
import math

@dataclass(eq=True, frozen=True)
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)

DIRECTIONS = {
    "<": Point(-1, 0),
    "^": Point(0, -1),
    ">": Point(1, 0),
    "v": Point(0, 1)
}

def parse_data(s):
    warehouse_map, instructions = s.strip().split("\n\n")

    warehouse = {}
    boxes = set()

    for row, line in enumerate(warehouse_map.splitlines()):
        for col, c in enumerate(line):
            loc = Point(col, row)
            if c == "#":
                warehouse[loc] = c
            else:
                warehouse[loc] = "."
                if c == "O":
                    boxes.add(loc)
                elif c == "@":
                    robot = loc

    return pmap(warehouse), pset(boxes), robot, instructions.replace("\n", "")

def part_1(data):
    warehouse, boxes, robot, instructions = data

    def move(boxes, robot, d):
        boxes_to_move = []
        p = robot + d
        while p in boxes:
            boxes_to_move.append(p)
            p = p + d
        if warehouse[p] == ".":
            robot = robot + d
            for box in boxes_to_move:
                boxes = boxes.remove(box)
            for box in boxes_to_move:
                boxes = boxes.add(box + d)

        return boxes, robot

    for i in instructions:
        boxes, robot = move(boxes, robot, DIRECTIONS[i])

    return sum(
        box.x +  100 * box.y
        for box in boxes
    )
        
test_data_1 = parse_data("""########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<""")

part_1(test_data_1)

2028

In [3]:
data = parse_data(open("input").read())

part_1(data)

1479679

## Part 2
This looks _really_ annoying.

In [8]:
import itertools

def parse_data_2(s):
    warehouse_map, instructions = s.strip().split("\n\n")

    warehouse = {}
    boxes = {}
    box_locations = {}
    box_ids = itertools.count()

    for row, line in enumerate(warehouse_map.splitlines()):
        for col, c in enumerate(line):
            loc1 = Point(col * 2, row)
            loc2 = Point(col * 2 + 1, row)
            if c == "#":
                warehouse[loc1] = c
                warehouse[loc2] = c
            else:
                warehouse[loc1] = "."
                warehouse[loc2] = "."
                if c == "O":
                    box_id = next(box_ids)
                    box_locations[loc1] = box_id
                    box_locations[loc2] = box_id
                    boxes[box_id] = [loc1, loc2]
                elif c == "@":
                    robot = loc1

    return pmap(warehouse), pmap(boxes), pmap(box_locations), robot, instructions.replace("\n", "")

def part_2(data):
    warehouse, boxes, box_locations, robot, instructions = data

    def move(boxes, box_locations, robot, d):
        boxes_to_move = []
        move_into = {robot + d}
        q = {robot + d}
        while q:
            p = q.pop()
            if p in boxes and boxes[p] not in boxes_to_move:
                b = box_locations[boxes[p]]
                q.add(b[0])
                q.add(b[1])
                move_into.add(b[0])
                move_into.add(b[1])
                boxes_to_move.append(boxes[p])
        if not any(warehouse[p] == "#" for p in move_into):
            robot = robot + d
            new_locations = {}
            for box in boxes_to_move:
                p1, p2 = boxes[box]
                new_locations[box] = [p1 + d, p2 + d]
                boxes = boxes.remove(box)
                box_locations = box_locations.remove(p1).remove(p2)
            for box in boxes_to_move:
                boxes = boxes.set(box + d, new_locations[box])
                for p in new_locations[box]:
                    box_locations = box_locations.set(p, box)

        return boxes, box_locations, robot

    for i in instructions:
        boxes, box_locations, robot = move(boxes, box_locations, robot, DIRECTIONS[i])

    return sum(
        boxes[box][0].x + 100 * boxes[box][0].y
        for box in boxes
    )

In [11]:
test_data_2 = parse_data_2("""#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^""")

In [12]:
part_2(test_data_2)

1020

In [13]:
test_data_3 = parse_data_2("""##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^""")

In [14]:
part_2(test_data_3)

9802

Nope.

Let's debug the small example.

In [36]:
def part_2(data, debug = False):
    warehouse, boxes, box_locations, robot, instructions = data

    def move(boxes, box_locations, robot, d):
        boxes_to_move = []
        move_into = {robot + d}
        q = {robot + d}
        while q:
            p = q.pop()
            if p in box_locations and box_locations[p] not in boxes_to_move:
                b = boxes[box_locations[p]]
                q.add(b[0] + d)
                q.add(b[1] + d)
                move_into.add(b[0] + d)
                move_into.add(b[1] + d)
                boxes_to_move.append(box_locations[p])

        if not any(warehouse[p] == "#" for p in move_into):
            robot = robot + d
            new_locations = {}
            for box in boxes_to_move:
                p1, p2 = boxes[box]
                new_locations[box] = [p1 + d, p2 + d]
                boxes = boxes.remove(box)
                box_locations = box_locations.remove(p1).remove(p2)
            for box in boxes_to_move:
                boxes = boxes.set(box, new_locations[box])
                for p in new_locations[box]:
                    box_locations = box_locations.set(p, box)

        return boxes, box_locations, robot

    for i in instructions:
        boxes, box_locations, robot = move(boxes, box_locations, robot, DIRECTIONS[i])
        if debug:
            print("Move", i)
            for row in range(max(p.y for p in warehouse) + 1):
                line = ""
                for col in range(max(p.x for p in warehouse) + 1):
                    p = Point(col, row)
                    if p == robot:
                        line += "@"
                    elif p in box_locations:
                        if boxes[box_locations[p]][0] == p:
                            line += "["
                        else:
                            line += "]"
                    else:
                        line += warehouse[p]
                print(line)
            print()

    return sum(
        boxes[box][0].x + 100 * boxes[box][0].y
        for box in boxes
    )

part_2(test_data_2, True)

Move <
##############
##......##..##
##..........##
##...[][]@..##
##....[]....##
##..........##
##############

Move v
##############
##......##..##
##..........##
##...[][]...##
##....[].@..##
##..........##
##############

Move v
##############
##......##..##
##..........##
##...[][]...##
##....[]....##
##.......@..##
##############

Move <
##############
##......##..##
##..........##
##...[][]...##
##....[]....##
##......@...##
##############

Move <
##############
##......##..##
##..........##
##...[][]...##
##....[]....##
##.....@....##
##############

Move ^
##############
##......##..##
##...[][]...##
##....[]....##
##.....@....##
##..........##
##############

Move ^
##############
##......##..##
##...[][]...##
##....[]....##
##.....@....##
##..........##
##############

Move <
##############
##......##..##
##...[][]...##
##....[]....##
##....@.....##
##..........##
##############

Move <
##############
##......##..##
##...[][]...##
##....[]....##
##...@......##
##..........##

618

Lots of squashed bugs there, but now looks good.

In [37]:
part_2(test_data_3)

9021

In [38]:
data_2 = parse_data_2(open("input").read())

part_2(data_2)

1509780

Finished!